In [1]:
!pip -q install --upgrade selenium beautifulsoup4 pandas
# If your environment has trouble fetching a driver automatically:
# !pip -q install webdriver-manager

- Runs shell commands from the notebook to install packages.

- ```selenium``` controls Chrome; ```beautifulsoup4``` parses HTML; ```pandas``` manages tables/CSV.

- You usually don’t need ```webdriver-manager``` because Selenium Manager (built into Selenium ≥4.6) auto-handles ChromeDriver.

- คำสั่งนี้ติดตั้งแพ็กเกจที่ต้องใช้: ```selenium``` สำหรับควบคุม Chrome, ```beautifulsoup4``` สำหรับอ่าน HTML, ```pandas``` สำหรับจัดการตาราง/บันทึก CSV

- ปกติไม่ต้องลง ```webdriver-manager``` เพราะ Selenium Manager จะจัดการไดรเวอร์ให้อัตโนมัติ

In [2]:
import re, time, random
from urllib.parse import quote_plus
from typing import List, Dict, Any, Optional

import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --------- Settings you can tweak ----------
KEYWORD              = "astaxanthin"  # <- change if you want
BASE_SEARCH_URL      = "https://www.lazada.co.th/catalog/?q={query}&page={page}"
HEADLESS             = False          # True recommended for Colab/servers
MAX_PAGES            = 50             # hard cap; we'll stop earlier if no more results
PER_PAGE_MIN_ITEMS   = 60             # scroll each page until ~this many items visible
PER_PAGE_MAX_SCROLLS = 20             # safety cap for scrolls per page
SCROLL_PAUSE_RANGE   = (1.0, 2.0)     # polite random delay between scrolls (seconds)
BETWEEN_PAGE_PAUSE   = (1.0, 2.0)     # pause between pages

# Robust-ish selectors (DOM can change; we keep fallbacks)
PRODUCT_ANCHOR_CSS = [
    "a[href*='/products/']",
    "a[data-qa-locator='product-item']",
]
NAME_CANDIDATES = [
    "div.RfADt",
    "a[title]",
    "div[data-qa-locator='product-title']",
]
PRICE_CANDIDATES = [
    "div.aBrP0",
    "span[data-qa-locator='product-price']",
    "span[aria-label*='price']",
]
SOLD_CANDIDATES = [
    "div._6uN7R",
]
RATING_CANDIDATES = [
    "span[data-qa-locator='rating-score']",
    "span[aria-label*='rating']",
]
REVIEWS_CANDIDATES = [
    "span[data-qa-locator='rating-total']",
]


- Imports core libs, Selenium, BeautifulSoup, and ```pandas```.

- Settings:

    - ```KEYWORD``` — search term.

    - ```BASE_SEARCH_URL``` — we’ll navigate page by page using ```page=1,2,3,....```

    - ```HEADLESS``` — set ```True``` to hide the browser window (good for Colab).

    - Pagination control: ```MAX_PAGES``` (hard limit), and we’ll break early if a page yields no new products.

    - Scrolling control per page: ```PER_PAGE_MIN_ITEMS```, ```PER_PAGE_MAX_SCROLLS```, ```SCROLL_PAUSE_RANGE```.

    - Selectors (```*_CANDIDATES```) give multiple fallbacks for product link/name/price/sold/rating/reviews to resist minor DOM changes.


- นำเข้าไลบรารีหลัก ๆ, Selenium, BeautifulSoup, และ ```pandas```

- การตั้งค่า:

    - ```KEYWORD``` — คำค้นหา

    - ```BASE_SEARCH_URL``` — ใช้พารามิเตอร์ ```page=``` เพื่อไล่หน้า 1,2,3,…

    - HEADLESS — ตั้ง ```True``` เพื่อไม่แสดงหน้าต่างเบราว์เซอร์ (เหมาะกับ Colab)

    - ควบคุมการเปลี่ยนหน้า: ```MAX_PAGES``` เป็นขีดจำกัด (หยุดก่อนถ้าหน้าที่ได้มาไม่มีสินค้าใหม่)

    - ควบคุมการเลื่อนในแต่ละหน้า: ```PER_PAGE_MIN_ITEMS```, ```PER_PAGE_MAX_SCROLLS```, ```SCROLL_PAUSE_RANGE```

    - Selector หลายแบบเพื่อกัน DOM เปลี่ยน

In [3]:
def clean_text(s: Optional[str]) -> str:
    if not s:
        return ""
    return " ".join(s.split())

def price_to_float(price_text: str) -> Optional[float]:
    if not price_text:
        return None
    t = price_text.replace(",", "")
    m = re.search(r"(\d+(?:\.\d+)?)", t)
    if not m:
        return None
    try:
        return float(m.group(1))
    except Exception:
        return None

def sold_to_number(sold_text: str) -> Optional[int]:
    """
    Convert 'ขายแล้ว 1.2พัน', 'Sold 1.2k', '10K+', '3 หมื่น', '2.5 ล้าน+' to an int.
    """
    if not sold_text:
        return None
    t = sold_text.lower().replace("+", "").strip()
    m = re.search(r"(\d+(?:\.\d+)?)", t)
    if not m:
        return None
    num = float(m.group(1))
    if any(k in t for k in ["k", "พัน"]):   return int(num * 1_000)
    if "หมื่น" in t:                        return int(num * 10_000)
    if "แสน" in t:                          return int(num * 100_000)
    if any(k in t for k in ["m", "ล้าน"]):  return int(num * 1_000_000)
    return int(num)


 -```clean_text``` normalizes whitespace.

 -```price_to_float``` extracts a numeric price from strings like ```฿1,290``` → ```1290.0```.

 -```sold_to_number``` maps Thai/English multipliers (k/พัน/หมื่น/แสน/m/ล้าน) to an integer estimate.



 -```clean_text``` จัดรูปแบบช่องว่างให้เรียบร้อย

 -```price_to_float``` ดึงตัวเลขจากข้อความราคา เช่น ```฿1,290``` → ```1290.0```

 -```sold_to_number``` แปลงตัวคูณไทย/อังกฤษ (k/พัน/หมื่น/แสน/m/ล้าน) เป็นจำนวนเต็มโดยประมาณ

In [12]:
def make_driver(headless: bool = HEADLESS) -> webdriver.Chrome:
    opts = Options()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1280,1600")
    opts.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/118.0.0.0 Safari/537.36"
    )
    service = Service()  # Selenium Manager will handle the driver
    return webdriver.Chrome(service=service, options=opts)

def maybe_accept_cookies_or_popups(driver: webdriver.Chrome) -> None:
    candidates = [
        "//button[contains(., 'ยอมรับ') or contains(., 'ตกลง') or contains(., 'Accept') or contains(., 'Agree')]",
        "//a[contains(., 'ยอมรับ') or contains(., 'Accept')]",
        "//button[contains(@class, 'close') or contains(@aria-label, 'close')]",
    ]
    for xp in candidates:
        try:
            el = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, xp)))
            el.click()
            time.sleep(0.3)
        except Exception:
            pass


 -```make_driver``` sets Chrome options and relies on Selenium Manager to find the right driver automatically.

 -```maybe_accept_cookies_or_popups``` tries common Thai/English buttons to dismiss cookie banners or modals; it’s okay if none are present.



 -```make_driver``` ตั้งค่า Chrome และให้ Selenium Manager จัดการไดรเวอร์ให้อัตโนมัติ

 -```maybe_accept_cookies_or_popups``` ลองกดปุ่มยอมรับ/ปิดป็อปอัป (ไทย/อังกฤษ) ถ้าไม่มี ก็ข้ามไป

In [13]:
def wait_for_results(driver: webdriver.Chrome, timeout: int = 20) -> None:
    def any_selector_present(drv):
        for css in PRODUCT_ANCHOR_CSS + NAME_CANDIDATES:
            try:
                if drv.find_elements(By.CSS_SELECTOR, css):
                    return True
            except Exception:
                pass
        return False
    WebDriverWait(driver, timeout).until(lambda d: any_selector_present(d))

def scroll_page_to_load(driver: webdriver.Chrome,
                        min_items: int = PER_PAGE_MIN_ITEMS,
                        max_scrolls: int = PER_PAGE_MAX_SCROLLS,
                        pause_range: tuple = SCROLL_PAUSE_RANGE) -> None:
    last_h = driver.execute_script("return document.body.scrollHeight")
    scrolls = 0
    while scrolls < max_scrolls:
        anchors = []
        for css in PRODUCT_ANCHOR_CSS:
            anchors.extend(driver.find_elements(By.CSS_SELECTOR, css))
        if len(anchors) >= min_items:
            break

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(*pause_range))

        new_h = driver.execute_script("return document.body.scrollHeight")
        if new_h == last_h:
            driver.execute_script("window.scrollBy(0, -400);")
            time.sleep(0.4)
            driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(random.uniform(*pause_range))
            new_h = driver.execute_script("return document.body.scrollHeight")
            if new_h == last_h:
                break

        last_h = new_h
        scrolls += 1


 - ```wait_for_results``` blocks until at least one known product/name selector is found.

 - ```scroll_page_to_load``` lazily scrolls the page to trigger lazy-loading; stops when enough anchors are seen or max_scrolls reached (with small random pauses).



 - ```wait_for_results``` รอจนกว่าจะเห็น element ที่เป็นสินค้า/ชื่อสินค้า

 - ```scroll_page_to_load``` เลื่อนหน้าจอเพื่อโหลดรายการเพิ่ม หยุดเมื่อมี anchor สินค้าเพียงพอหรือครบจำนวนครั้งที่กำหนด (พักแบบสุ่มเล็กน้อย)

In [14]:
def soup_select_first_text(node: BeautifulSoup, css_list: List[str]) -> str:
    for css in css_list:
        try:
            found = node.select_one(css)
            if found:
                t = clean_text(found.get_text())
                if t:
                    return t
        except Exception:
            pass
    return ""

def parse_page_cards(page_html: str) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(page_html, "html.parser")

    # Collect anchors
    anchors = []
    for css in PRODUCT_ANCHOR_CSS:
        anchors.extend(soup.select(css))

    # Deduplicate by href and normalize to absolute URLs
    seen, uniq = set(), []
    for a in anchors:
        href = a.get("href") or ""
        if not href:
            continue
        if href.startswith("//"): href = "https:" + href
        elif href.startswith("/"): href = "https://www.lazada.co.th" + href
        if href in seen:
            continue
        seen.add(href)
        uniq.append((a, href))

    rows = []
    for a, href in uniq:
        card = a
        for _ in range(3):  # go up a bit to get the product card container
            if card.parent:
                card = card.parent
            else:
                break

        name = clean_text(a.get("title")) or clean_text(a.get_text())
        if not name:
            name = soup_select_first_text(card, NAME_CANDIDATES)

        price_text   = soup_select_first_text(card, PRICE_CANDIDATES)
        sold_text    = soup_select_first_text(card, SOLD_CANDIDATES)
        rating_text  = soup_select_first_text(card, RATING_CANDIDATES)
        reviews_text = soup_select_first_text(card, REVIEWS_CANDIDATES)

        if not price_text:
            price_text = soup_select_first_text(soup, PRICE_CANDIDATES)
        if not sold_text:
            sold_text  = soup_select_first_text(soup, SOLD_CANDIDATES)

        rows.append({
            "name": name,
            "price_text": price_text,
            "price": price_to_float(price_text),
            "sold_text": sold_text,
            "sold_est": sold_to_number(sold_text) if sold_text else None,
            "rating_text": rating_text,
            "reviews_text": reviews_text,
            "product_url": href,
        })

    rows = [r for r in rows if r["name"] and r["product_url"]]
    return rows


 - Finds all product anchors, dedupes by ```href```, normalizes URLs, then for each card extracts ```name```, ```price```, ```sold```, ```rating```, ```reviews``` with fallbacks.

 - Converts text fields to numeric ```price``` and ```sold_est``` when possible.


 - ดึงลิงก์สินค้าทั้งหมด, ลบซ้ำด้วย ```href```, ทำ URL ให้เป็นแบบสมบูรณ์ แล้วดึง ```name```, ```price```, ```sold```, ```rating```, ```reviews``` พร้อม fallback

 - แปลง ```price_text``` และ ```sold_text``` เป็นตัวเลข (```price```, ```sold_est```) เมื่อทำได้

In [15]:
def scrape_lazada_all_pages(keyword: str,
                            max_pages: int = MAX_PAGES,
                            per_page_min_items: int = PER_PAGE_MIN_ITEMS,
                            per_page_max_scrolls: int = PER_PAGE_MAX_SCROLLS,
                            headless: bool = HEADLESS) -> pd.DataFrame:
    driver = make_driver(headless=headless)
    all_rows: List[Dict[str, Any]] = []
    seen_urls = set()

    try:
        for page in range(1, max_pages + 1):
            url = BASE_SEARCH_URL.format(query=quote_plus(keyword), page=page)
            driver.get(url)
            maybe_accept_cookies_or_popups(driver)

            try:
                wait_for_results(driver, timeout=20)
            except Exception:
                # If page loads but shows no products, stop
                print(f"[Page {page}] No results detected. Stopping.")
                break

            # Scroll within this page to load items
            scroll_page_to_load(
                driver,
                min_items=per_page_min_items,
                max_scrolls=per_page_max_scrolls,
                pause_range=SCROLL_PAUSE_RANGE,
            )

            html = driver.page_source
            rows = parse_page_cards(html)

            # Keep only new URLs to detect end of pagination
            new_rows = []
            for r in rows:
                u = r["product_url"]
                if u not in seen_urls:
                    seen_urls.add(u)
                    new_rows.append(r)

            print(f"[Page {page}] parsed {len(rows)} rows, new {len(new_rows)}")

            if not new_rows:
                # No new items on this page → likely end of pagination or duplicate page
                print(f"[Page {page}] No new products found. Stopping.")
                break

            all_rows.extend(new_rows)

            # polite pause before next page
            time.sleep(random.uniform(*BETWEEN_PAGE_PAUSE))

        df = pd.DataFrame(all_rows)
        if not df.empty:
            df = df.drop_duplicates(subset=["product_url"]).reset_index(drop=True)
        return df

    finally:
        driver.quit()


 - Opens Chrome once, then loops ```page=1..MAX_PAGES```.

 - Builds the search URL with ```q=<keyword>&page=<page>```.

 - Waits for results, scrolls each page to load all items, parses them, and keeps only new URLs (helps detect the last page).

 - Stops when a page returns no new products or when ```MAX_PAGES``` is reached.

 - Returns a deduplicated DataFrame of all pages.



 - เปิด Chrome หนึ่งครั้ง แล้ววนหน้า ```page=1..MAX_PAGES```

 - สร้าง URL ```q=<keyword>&page=<page>```

 - รอผลลัพธ์, เลื่อนหน้าภายในหน้านั้น เพื่อโหลดสินค้าทั้งหมด, แปลง HTML เป็นแถวข้อมูล และ เก็บเฉพาะ URL ใหม่ (เพื่อเช็คว่าไปสุดหน้าสุดท้ายแล้วหรือยัง)

 - หยุดเมื่อไม่พบสินค้าใหม่ในหน้านั้น หรือครบ ```MAX_PAGES```

 - คืนค่า DataFrame ที่ลบซ้ำแล้ว รวมทุกหน้า

In [16]:
df = scrape_lazada_all_pages(
    keyword=KEYWORD,
    max_pages=MAX_PAGES,
    per_page_min_items=PER_PAGE_MIN_ITEMS,
    per_page_max_scrolls=PER_PAGE_MAX_SCROLLS,
    headless=HEADLESS,
)

print(f"Total unique products: {len(df)}")
display(df.head(10))

out_path = f"lazada_{KEYWORD}_allpages.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print("Saved to:", out_path)


[Page 1] parsed 43 rows, new 43
[Page 2] parsed 43 rows, new 13
[Page 3] parsed 43 rows, new 34
[Page 4] parsed 43 rows, new 38
[Page 5] parsed 43 rows, new 40
[Page 6] parsed 43 rows, new 40
[Page 7] parsed 43 rows, new 40
[Page 8] parsed 43 rows, new 40
[Page 9] parsed 43 rows, new 40
[Page 10] parsed 43 rows, new 39
[Page 11] parsed 43 rows, new 40
[Page 12] parsed 43 rows, new 40
[Page 13] parsed 43 rows, new 40
[Page 14] parsed 43 rows, new 40
[Page 15] parsed 43 rows, new 40
[Page 16] parsed 43 rows, new 40
[Page 17] parsed 43 rows, new 40
[Page 18] parsed 43 rows, new 40
[Page 19] parsed 43 rows, new 40
[Page 20] parsed 43 rows, new 40
[Page 21] parsed 43 rows, new 40
[Page 22] parsed 43 rows, new 40
[Page 23] parsed 43 rows, new 40
[Page 24] parsed 43 rows, new 40
[Page 25] parsed 43 rows, new 40
[Page 26] parsed 43 rows, new 40
[Page 27] parsed 43 rows, new 40
[Page 28] parsed 43 rows, new 40
[Page 29] parsed 43 rows, new 40
[Page 30] parsed 43 rows, new 40
[Page 31] parsed 43

,name,price_text,price,sold_text,sold_est,rating_text,reviews_text,product_url
0,Dr.PONG Astaxanthin 6 mg AstaREAL from Japan ด...,฿379.00,379.00,100.4K ชิ้น(29581)ปทุมธานี,100400.0,,,https://www.lazada.co.th/products/pdp-i3640665...
1,Dr.PONG Special Set Astaxanthin แอสตาแซนธิน 3 ...,"฿1,099.00",1099.00,3.5K ชิ้น(1584)ปทุมธานี,3500.0,,,https://www.lazada.co.th/products/pdp-i5372715...
2,Dr.PONG เซตสุดคุ้ม 008 : เซตอาหารเสริมขายดี (ว...,฿849.00,849.00,784 ชิ้น(361)ปทุมธานี,784.0,,,https://www.lazada.co.th/products/pdp-i5556217...
3,( Pack 2 ) VISTRA ASTAXANTHIN 6 MG PLUS VITAMI...,"฿1,232.00",1232.00,3.4K ชิ้น(1388)นนทบุรี,3400.0,,,https://www.lazada.co.th/products/pdp-i5014701...
4,FITWHEY ASTAXANTHIN 6MG + COQ10 (30 SOFTGELS) ...,฿199.00,199.00,7.4K ชิ้น(2373)สมุทรปราการ,7400.0,,,https://www.lazada.co.th/products/pdp-i5067987...
5,(EXP: 23/11/2025) Astaxanthin 6 MG + CoQ10 แอส...,฿99.00,99.00,116 ชิ้น(60)สมุทรปราการ,116.0,,,https://www.lazada.co.th/products/pdp-i4898321...
6,[ร้านบริษัท]INZENT (เซต 2 กระปุก) คู่จิ้น ASTA...,฿509.04,509.04,นครราชสีมา,NaN,,,https://www.lazada.co.th/products/pdp-i5855061...
7,บีลีฟ แอสตาแซนธิน 8 มก. ( แอสตาอิงฟ้า ) Beleaf...,"฿2,112.32",2112.32,นนทบุรี,NaN,,,https://www.lazada.co.th/products/pdp-i5855146...
8,KIRKLAND 24mg Natural Astaxanthin Capsules สนั...,฿223.00,223.00,255 ชิ้น(79)เชียงใหม่,255.0,,,https://www.lazada.co.th/products/pdp-i5688665...
9,Astaxanthin 12 mg Softgels สนับสนุนสุขภาพภูมิค...,฿197.00,197.00,1.2K ชิ้น(423)เชียงใหม่,1200.0,,,https://www.lazada.co.th/products/pdp-i5661343...


Saved to: lazada_astaxanthin_allpages.csv


 - Runs the full crawl over every page for the keyword, shows a preview, and saves to ```lazada_astaxanthin_allpages.csv``` (UTF-8 with BOM so Thai displays correctly in Excel).


 - รันการดึงข้อมูลครบทุกหน้า ตามคำค้นที่กำหนด แสดงตัวอย่างตาราง และบันทึกเป็น ```lazada_astaxanthin_allpages.csv``` (เข้ารหัส UTF-8 พร้อม BOM เพื่อให้ Excel อ่านภาษาไทยได้ถูกต้อง)

### Notes & troubleshooting



 - If you see “No results detected” on page 1, Lazada’s DOM or blocking behavior may have changed; increase timeouts or adjust selectors (```PRODUCT_ANCHOR_CSS```, etc.).

 - If you get very few results, increase ```PER_PAGE_MIN_ITEMS``` or ```PER_PAGE_MAX_SCROLLS```.

 - On hosted environments (Colab), set ```HEADLESS = True```.



 - ถ้าขึ้น “No results detected” ตั้งแต่หน้าแรก อาจเกิดจาก DOM/การบล็อกของเว็บ ให้เพิ่มเวลา timeout หรือปรับ selector (```PRODUCT_ANCHOR_CSS``` ฯลฯ)

 - ถ้าข้อมูลน้อย ให้เพิ่ม ```PER_PAGE_MIN_ITEMS``` หรือ ```PER_PAGE_MAX_SCROLLS```

 - ถ้าใช้ Colab แนะนำ ```HEADLESS = True```